In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import yfinance as yf
import pandas_ta as ta
data = yf.download(tickers = 'BTC-EUR', start = '2024-03-11',end = '2024-06-16', interval='1H')
data

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2024-03-11 00:00:00,63078.578125,63078.578125,61618.460938,62232.707031,62232.707031,0
2024-03-11 01:00:00,62265.421875,62489.234375,61930.832031,62445.128906,62445.128906,243064832
2024-03-11 02:00:00,62467.898438,62650.320312,62465.605469,62489.285156,62489.285156,30777344
2024-03-11 03:00:00,62485.445312,62716.468750,62481.199219,62710.578125,62710.578125,85745664
2024-03-11 04:00:00,62713.414062,62838.511719,62641.988281,62682.476562,62682.476562,0
...,...,...,...,...,...,...
2024-06-15 15:00:00,61752.792969,61956.761719,61751.343750,61839.183594,61839.183594,0
2024-06-15 16:00:00,61839.183594,61872.554688,61728.785156,61728.785156,61728.785156,0
2024-06-15 17:00:00,61728.785156,61820.007812,61677.515625,61791.796875,61791.796875,0


In [2]:
# Adding indicators
data['RSI']=ta.rsi(data.Close, length=15)
data['EMAF']=ta.ema(data.Close, length=20)
data['EMAM']=ta.ema(data.Close, length=100)
data['EMAS']=ta.ema(data.Close, length=150)
data['ATR'] =data.ta.atr(length=20)

# data["CCI"] = ta.cci(data.High, data.Low, data.Close, length=16)
# data["AO"] = ta.ao(data.High, data.Low)
# data["MOM"] = ta.mom(data.Close, length=16)
# a = ta.macd(data.Close)
# data = data.join(a)
# data["BOP"] = ta.bop(data.Open, data.High, data.Low, data.Close, length=16)
# data["RVI"] = ta.rvi(data.Close)
# a = ta.dm(data.High, data.Low, length=16)
# data = data.join(a)
# a = ta.stoch(data.High, data.Low, data.Close)
# data = data.join(a)
# a = ta.stochrsi(data.Close, length=16)
# data = data.join(a)
# data["WPR"] = ta.willr(data.High, data.Low, data.Close, length=16)

data['Average'] = data.ta.midprice(length=1) #midprice
data['MA40'] = ta.sma(data.Close,length=40)
data['MA80'] = ta.sma(data.Close,length=80)
data['MA160'] = ta.sma(data.Close,length=160)

from scipy.stats import linregress
def get_slope(array):
    y = np.array(array)
    x = np.arange(len(y))
    slope, intercept, r_value, p_value, std_err = linregress(x,y)
    return slope

#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
backrollingN = 6
#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
data['slopeMA40'] = data['MA40'].rolling(window=backrollingN).apply(get_slope, raw=True)
data['slopeMA80'] = data['MA80'].rolling(window=backrollingN).apply(get_slope, raw=True)
data['slopeMA160'] = data['MA160'].rolling(window=backrollingN).apply(get_slope, raw=True)
data['AverageSlope'] = data['Average'].rolling(window=backrollingN).apply(get_slope, raw=True)
data['RSISlope'] = data['RSI'].rolling(window=backrollingN).apply(get_slope, raw=True)

data['Target'] = data['Adj Close']-data.Open
data['Target'] = data['Target'].shift(-1)

data['TargetClass'] = [1 if data.Target[i]>0 else 0 for i in range(len(data))]

data['TargetNextClose'] = data['Adj Close'].shift(-1)

data.dropna(inplace=True)
data.reset_index(inplace = True)
data.drop(['Volume', 'Close', 'Datetime'], axis=1, inplace=True)

C:\Users\hozay\AppData\Local\Temp\ipykernel_8396\903671938.py:47: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  data['TargetClass'] = [1 if data.Target[i]>0 else 0 for i in range(len(data))]


In [3]:
data_set = data.iloc[:, 0:21]#.values
pd.set_option('display.max_columns', None)
data_set.head(20)

#print(data_set.shape)
#print(data.shape)
#print(type(data_set))

,Open,High,Low,Adj Close,RSI,EMAF,EMAM,EMAS,ATR,Average,MA40,MA80,MA160,slopeMA40,slopeMA80,slopeMA160,AverageSlope,RSISlope,Target,TargetClass,TargetNextClose
0,62914.246094,63009.421875,62695.683594,62742.605469,59.463416,61898.072394,63042.542842,64089.993188,606.431302,62852.552734,61814.603320,62747.299072,64403.929639,-23.876945,-51.834766,0.899155,149.946987,0.487516,19.460938,1,62690.726562
1,62671.265625,62734.675781,62455.660156,62690.726562,58.792071,61973.563267,63035.576183,64071.459855,590.453631,62595.167969,61797.592285,62704.967920,64404.222144,-19.170915,-48.790063,0.738092,50.585714,0.209066,-12.402344,0,62803.500000
2,62815.902344,63187.218750,62612.109375,62803.500000,59.847875,62052.604860,63030.980615,64054.665685,589.686264,62899.664062,61779.323535,62664.941553,64401.503638,-17.718117,-45.942741,0.081525,33.998940,-0.077595,43.496094,1,62750.925781
3,62707.429688,63017.070312,62547.777344,62750.925781,59.091639,62119.111615,63025.434975,64037.397607,583.665452,62782.423828,61757.846094,62633.633838,64387.858228,-17.180413,-42.227729,-2.266788,12.703069,-0.064387,-970.191406,0,61774.175781
4,62744.367188,62744.367188,61774.175781,61774.175781,47.215672,62086.260583,63000.657565,64007.421159,603.323237,62259.271484,61717.005762,62589.055908,64367.351733,-21.563390,-40.213361,-6.626950,-71.984989,-1.980649,586.093750,1,62086.187500
5,61500.093750,62192.539062,61372.093750,62086.187500,50.612772,62086.253623,62982.549247,63981.974355,614.181207,61782.316406,61689.183496,62550.617432,64345.143677,-25.438175,-38.927199,-11.948184,-185.031752,-2.278247,384.511719,1,62409.941406
6,62025.429688,62507.273438,61889.234375,62409.941406,53.910484,62117.081031,62971.210478,63961.152727,614.374131,62198.253906,61676.870996,62526.161084,64326.872119,-26.139054,-36.616699,-16.466757,-167.421875,-1.828263,-142.562500,0,62269.031250
7,62411.593750,62510.179688,62267.472656,62269.031250,52.282519,62131.552480,62957.305939,63938.740522,595.787893,62388.826172,61666.757031,62500.339746,64307.296069,-23.816574,-33.824736,-19.320121,-136.675837,-1.427804,419.718750,1,62633.132812
8,62213.414062,62907.429688,62165.496094,62633.132812,55.964005,62179.322036,62950.886669,63921.447705,603.096255,62536.462891,61674.570996,62471.817676,64287.272290,-16.555262,-31.419590,-20.039092,-12.148661,0.081717,298.578125,1,62984.671875
9,62686.093750,63266.976562,62686.093750,62984.671875,59.218775,62256.022020,62951.555683,63909.040078,604.633845,62976.535156,61692.621777,62440.703174,64271.434570,-5.024897,-28.685265,-19.222172,172.552288,2.126893,-133.968750,0,62782.929688


In [4]:
#Target column Categories
#y =[1 if data.Open[i]>data.Close[i] else 0 for i in range(0, len(data))]
#yi = [data.Open[i]-data.Close[i] for i in range(0, len(data))]
#print(yi)
#print(len(yi))

In [5]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range=(0,1))
data_set_scaled = sc.fit_transform(data_set)
print(data_set_scaled)

[[0.71136    0.69473034 0.70820325 ... 0.55013885 1.         0.69652837]
 [0.69357801 0.67445389 0.69064451 ... 0.54371996 0.         0.70478489]
 [0.70416293 0.70785188 0.70208944 ... 0.55498076 1.         0.70093576]
 ...
 [0.63268394 0.61082875 0.63747048 ... 0.5589122  1.         0.63071472]
 [0.62460468 0.60695076 0.6337199  ... 0.52430198 0.         0.62296954]
 [0.62943588 0.60509046 0.63387421 ... 0.53687929 0.         0.62013138]]


In [6]:
# multiple feature from data provided to the model
X = []
#print(data_set_scaled[0].size)
#data_set_scaled=data_set.values
backcandles = 100
print(data_set_scaled.shape[0])
for j in range(18):#data_set_scaled[0].size):#2 columns are target not X
    X.append([])
    for i in range(backcandles, data_set_scaled.shape[0]):#backcandles+2
        X[j].append(data_set_scaled[i-backcandles:i, j])

#move axis from 0 to position 2
X=np.moveaxis(X, [0], [2])

#Erase first elements of y because of backcandles to match X length
#del(yi[0:backcandles])
#X, yi = np.array(X), np.array(yi)
# Choose -1 for last column, classification else -2...
X, yi =np.array(X), np.array(data_set_scaled[backcandles:,-1])
y=np.reshape(yi,(len(yi),1))
#y=sc.fit_transform(yi)
#X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
#print(X)
print(X.shape)
#print(y)
print(y.shape)

2159
(2059, 100, 18)
(2059, 1)


In [7]:
#also comprehensions for X
#X = np.array([data_set_scaled[i-backcandles:i,:4].copy() for i in range(backcandles,len(data_set_scaled))])
#print(X)
#print(X.shape)

In [8]:
# split data into train test sets
splitlimit = int(len(X)*0.8)
print(splitlimit)
X_train, X_test = X[:splitlimit], X[splitlimit:]
y_train, y_test = y[:splitlimit], y[splitlimit:]
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)
print(y_train)

1647
(1647, 100, 18)
(412, 100, 18)
(1647, 1)
(412, 1)
[[0.5159362 ]
 [0.54290531]
 [0.56495111]
 ...
 [0.68730664]
 [0.68230696]
 [0.66482842]]


In [9]:
print(X_train.shape[2])

18


In [10]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import LSTM, Dense, Input, Activation, concatenate, TimeDistributed, Dropout
from tensorflow.keras.optimizers import Adam

lstm_input = Input(shape=(backcandles, X_train.shape[2]), name='lstm_input')
inputs = LSTM(150, name='first_layer')(lstm_input)
inputs = Dense(1, name='dense_layer')(inputs)
output = Activation('linear', name='output')(inputs)
model = Model(inputs=lstm_input, outputs=output)
adam = Adam()
model.compile(optimizer=adam, loss='mse')
model.fit(x=X_train, y=y_train, batch_size=15, epochs=30, shuffle=True, validation_split = 0.1)

Epoch 1/30
99/99 ━━━━━━━━━━━━━━━━━━━━ 3s 23ms/step - loss: 0.0459 - val_loss: 0.0015
Epoch 2/30
99/99 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - loss: 0.0028 - val_loss: 0.0015
Epoch 3/30
99/99 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - loss: 0.0020 - val_loss: 0.0010
Epoch 4/30
99/99 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - loss: 0.0018 - val_loss: 0.0010
Epoch 5/30
99/99 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - loss: 0.0019 - val_loss: 9.4631e-04
Epoch 6/30
99/99 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - loss: 0.0016 - val_loss: 0.0011
Epoch 7/30
99/99 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - loss: 0.0016 - val_loss: 0.0012
Epoch 8/30
99/99 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - loss: 0.0018 - val_loss: 0.0013
Epoch 9/30
99/99 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - loss: 0.0018 - val_loss: 8.5295e-04
Epoch 10/30
99/99 ━━━━━━━━━━━━━━━━━━━━ 2s 21ms/step - loss: 0.0016 - val_loss: 8.7637e-04
Epoch 11/30
99/99 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - loss: 0.0015 - val_loss: 0.0013
Epoch 12/30
99/99 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step

In [ ]:
backcandles = X_train.shape[1]
feature_count = X_train.shape[2]
# Define the input shape
lstm_input = Input(batch_shape=(1, backcandles, feature_count), name='lstm_input')

# LSTM layer with stateful=True
lstm = LSTM(250, stateful=True, return_sequences=False)(lstm_input)

# Dense layer for output
output = Dense(1, activation='linear', name='output')(lstm)

# Define the model
future_model = Model(inputs=lstm_input, outputs=output)
adam = Adam()
future_model.compile(optimizer=adam, loss='mse')

# Fit the model (note the batch_size=1 due to stateful LSTM)
future_model.fit(x=X_train, y=y_train, batch_size=1, epochs=30, shuffle=False, validation_split=0.1)

# # Define input shape dimensions
# backcandles = X_train.shape[1]
# feature_count = X_train.shape[2]

# # Define LSTM input layer
# lstm_input = Input(batch_shape=(1, backcandles, feature_count), name='lstm_input')

# # LSTM layers with stateful=True and return_sequences=True for deeper learning
# lstm = LSTM(150, stateful=True, return_sequences=True)(lstm_input)
# lstm = Dropout(0.2)(lstm)  # Adding dropout for regularization
# lstm = LSTM(100, stateful=True, return_sequences=False)(lstm)

# # Dense layer for output prediction
# output = Dense(1, activation='linear', name='output')(lstm)

# # Define the model
# future_model = Model(inputs=lstm_input, outputs=output)
# adam = Adam(learning_rate=0.001)  # Adjusted learning rate
# future_model.compile(optimizer=adam, loss='mse')

# # Fit the model
# future_model.fit(x=X_train, y=y_train, batch_size=1, epochs=10, shuffle=True, validation_split=0.1)


Epoch 1/30
1482/1482 ━━━━━━━━━━━━━━━━━━━━ 32s 21ms/step - loss: 0.0069 - val_loss: 0.0131
Epoch 2/30
1482/1482 ━━━━━━━━━━━━━━━━━━━━ 30s 20ms/step - loss: 0.0038 - val_loss: 0.0080
Epoch 3/30
1482/1482 ━━━━━━━━━━━━━━━━━━━━ 30s 20ms/step - loss: 0.0035 - val_loss: 0.0062
Epoch 4/30
1482/1482 ━━━━━━━━━━━━━━━━━━━━ 30s 20ms/step - loss: 0.0034 - val_loss: 0.0045
Epoch 5/30
1482/1482 ━━━━━━━━━━━━━━━━━━━━ 30s 20ms/step - loss: 0.0038 - val_loss: 0.0021
Epoch 6/30
1482/1482 ━━━━━━━━━━━━━━━━━━━━ 31s 21ms/step - loss: 0.0035 - val_loss: 0.0015
Epoch 7/30
1482/1482 ━━━━━━━━━━━━━━━━━━━━ 30s 20ms/step - loss: 0.0032 - val_loss: 0.0019
Epoch 8/30
1482/1482 ━━━━━━━━━━━━━━━━━━━━ 30s 20ms/step - loss: 0.0029 - val_loss: 0.0025
Epoch 9/30
1482/1482 ━━━━━━━━━━━━━━━━━━━━ 30s 20ms/step - loss: 0.0027 - val_loss: 0.0029
Epoch 10/30
1482/1482 ━━━━━━━━━━━━━━━━━━━━ 30s 20ms/step - loss: 0.0024 - val_loss: 0.0070
Epoch 11/30
 301/1482 ━━━━━━━━━━━━━━━━━━━━ 23s 20ms/step - loss: 0.0023

In [ ]:
y_pred = model.predict(X_test)
#y_pred=np.where(y_pred > 0.43, 1,0)
for i in range(10):
    print(y_pred[i], y_test[i])

In [ ]:
# # Function to reset LSTM states
# def reset_model_states(model):
#     for layer in model.layers:
#         if isinstance(layer, LSTM):
#             layer.reset_states()

# # Function to predict future values
# def predict_future(model, last_data, num_predictions):
#     predictions = []
#     current_input = last_data

#     for _ in range(num_predictions):
#         reset_model_states(model)
#         prediction = model.predict(current_input, batch_size=1)
#         predictions.append(prediction[0, 0])
        
#         # Prepare the new input by removing the first time step and appending the prediction
#         # Expand the prediction to match the feature dimensions
#         prediction_expanded = np.full((1, 1, feature_count), prediction[0, 0])
#         new_input = np.concatenate([current_input[:, 1:, :], prediction_expanded], axis=1)
#         current_input = new_input

#     return np.array(predictions)

# # Example usage to predict future values
# last_sequence = X_train[-1:]  # Get the last sequence from training set
# num_predictions = 10  # Number of future predictions
# future_predictions = predict_future(future_model, last_sequence, num_predictions)
# print("Future Predictions:", future_predictions)
##########################################################

def reset_model_states(model):
    for layer in future_model.layers:
        if isinstance(layer, LSTM):
            layer.reset_states()

# Predicting future values
def predict_future(future_model, last_data, num_predictions):
    predictions = []
    current_input = last_data

    for _ in range(num_predictions):
        # Reset states before each prediction sequence
        reset_model_states(future_model)
        # Make a prediction
        prediction = future_model.predict(current_input, batch_size=1)
        predictions.append(prediction[0, 0])
        # Prepare the new input by removing the first time step and appending the prediction
        # Expand the prediction to match the feature dimensions
        prediction_expanded = np.full((1, 1, feature_count), prediction[0, 0])
        new_input = np.concatenate([current_input[:, 1:, :], prediction_expanded], axis=1)
        current_input = new_input

    return np.array(predictions)

# Get the last sequence from the training set for initial prediction
last_sequence = X_train[-1:]  # shape (1, backcandles, feature_count)

# Number of future predictions you want to make
num_predictions = 10

# Predict future values
future_predictions = predict_future(future_model, last_sequence, num_predictions)

print("Future Predictions:", future_predictions)
#####################################################
# # Function to reset the states of LSTM layers
# def reset_model_states(model):
#     for layer in model.layers:
#         if isinstance(layer, LSTM):
#             layer.reset_states()

# # Predicting future values
# def predict_future(model, last_data, num_predictions):
#     predictions = []
#     current_input = last_data

#     for _ in range(num_predictions):
#         # Reset states before each prediction sequence
#         reset_model_states(model)
#         # Make a prediction
#         prediction = model.predict(current_input, batch_size=1)
#         predictions.append(prediction[0, 0])
#         # Prepare the new input by removing the first time step and appending the prediction
#         # Expand the prediction to match the feature dimensions
#         prediction_expanded = np.full((1, 1, feature_count), prediction[0, 0])
#         new_input = np.concatenate([current_input[:, 1:, :], prediction_expanded], axis=1)
#         current_input = new_input

#     return np.array(predictions)

# # Get the last sequence from the training set for initial prediction
# last_sequence = X_train[-1:]  # shape (1, backcandles, feature_count)

# # Number of future predictions you want to make
# num_predictions = 10

# # Predict training values
# train_predictions = model.predict(X_train, batch_size=1)

# # Predict future values
# future_predictions = predict_future(model, last_sequence, num_predictions)

# # Combine training and future predictions
# all_predictions = np.concatenate([train_predictions.flatten(), future_predictions])

# print("All Predictions (Training + Future):", all_predictions[-11:-1])



In [ ]:
# # Plotting the results
# import matplotlib.pyplot as plt

# plt.figure(figsize=(12, 6))
# plt.plot(range(len(y_train)), y_train, label='True Values', color='black')
# plt.plot(range(len(y_train)), train_predictions.flatten(), label='Training Predictions', color='green')
# plt.plot(range(len(y_train), len(y_train) + num_predictions), future_predictions, label='Future Predictions')
# plt.xlabel('Time Step')
# plt.ylabel('Value')
# plt.legend()
# plt.show()

In [ ]:
future_predictions=np.insert(future_predictions,0,y_pred[-1])

In [ ]:
num_predictions = len(future_predictions)
print(num_predictions)

In [ ]:
# plt.figure(figsize=(16,8))
# plt.plot(y_test, color = 'black', label = 'Test')
# plt.plot(y_pred, color = 'green', label = 'pred')
# plt.legend()
# plt.show()

# Define x_values for plotting
x_values = range(len(y_test) + num_predictions)

# Create a figure and axis with a larger size for better visibility
plt.figure(figsize=(20, 8))

# Plot test and prediction data for X_test
plt.plot(x_values[-(len(y_test) + num_predictions):-num_predictions], y_test, color='black', label='True Values (Test)', linestyle='-', marker='o', markersize=5)
plt.plot(x_values[-(len(y_test) + num_predictions):-num_predictions], y_pred, color='blue', label='Predicted (X_test)', linestyle='--', marker='x', markersize=5)

# Plot future predictions
plt.plot(x_values[-num_predictions:], future_predictions, color='red', label='Future Predictions', linestyle='--', marker='x', markersize=5)

# Add title and labels
plt.title('Test vs Predicted Values')
plt.xlabel('Time')
plt.ylabel('Values')

# Add grid for better readability
plt.grid(True)

# Set x ticks to show all data points
plt.xticks(x_values[-(len(y_test) + num_predictions):-1])

# Add legend
plt.legend()

# Show the plot
plt.show()

In [ ]:
# Number of hours to display
num_hours =60
num_predictions = len(future_predictions)

# Calculate the start index for x_values based on the length of y_test and num_hours
start_index = len(y_test) - num_hours

# Define x_values for plotting
x_values = range(start_index, start_index + len(y_test[-num_hours:]) + num_predictions)

# Create a figure and axis with a larger size for better visibility
plt.figure(figsize=(20, 8))

# Plot test and prediction data for X_test
plt.plot(x_values[:len(y_test[-num_hours-1:])], y_test[-num_hours-1:], color='black', label='True Values (Test)', linestyle='-', marker='o', markersize=5)
plt.plot(x_values[:len(y_pred[-num_hours-1:])], y_pred[-num_hours-1:], color='blue', label='Predicted (X_test)', linestyle='--', marker='x', markersize=5)

# Adjust future_x_values to have the same length as future_predictions
future_x_values = range(start_index + len(y_test[-num_hours:]), start_index + len(y_test[-num_hours:]) + len(future_predictions))

# Plot future predictions
plt.plot(future_x_values, future_predictions, color='red', label='Future Predictions', linestyle='--', marker='x', markersize=5)

# Add title and labels
plt.title('Test vs Predicted Values (Last 30 Hours)')
plt.xlabel('Time')
plt.ylabel('Values')

# Add grid for better readability
plt.grid(True)

# Set x ticks to show all data points
plt.xticks(x_values)

# Add legend
plt.legend()

# Show the plot
plt.show()